# Naive RAG 간단구성

- 관련 패키지 임포트
- 활용 LLM api정보 설정
- RAG 파이프라인 글로벌 세팅 설정
- 기준 데이터셋 로드
- 기준 데이터셋 전처리
- 벡터스토어인덱스 설정
- 쿼리 엔진 설정

In [1]:
# 관련 패키지 임포트
import os
from datasets import load_dataset
from llama_index.core import Document
from llama_index.core import VectorStoreIndex
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
import nest_asyncio

nest_asyncio.apply()

In [2]:
# RAG 파이프라인 글로벌 설정
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)

Settings.llm=OpenAI(model='gpt-3.5-turbo',temperature=0)

In [3]:
# 기준 데이터셋 로드 - 
ds = load_dataset("lcw99/wikipedia-korean-20240501", split='train[-10:]')

README.md:   0%|          | 0.00/475 [00:00<?, ?B/s]

train-00000-of-00007.parquet:   0%|          | 0.00/468M [00:00<?, ?B/s]

train-00001-of-00007.parquet:   0%|          | 0.00/289M [00:00<?, ?B/s]

train-00002-of-00007.parquet:   0%|          | 0.00/232M [00:00<?, ?B/s]

train-00003-of-00007.parquet:   0%|          | 0.00/189M [00:00<?, ?B/s]

train-00004-of-00007.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

train-00005-of-00007.parquet:   0%|          | 0.00/186M [00:00<?, ?B/s]

train-00006-of-00007.parquet:   0%|          | 0.00/198M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/515425 [00:00<?, ? examples/s]

In [4]:
ds


Dataset({
    features: ['title', 'section_titles', 'section_texts', 'text'],
    num_rows: 10
})

In [5]:
# 기준 데이터셋 전처리
data = ds.to_pandas()[['title', 'text']]
data.drop_duplicates(subset='text', keep='first', inplace=True)
data.head()

,title,text
0,학도요,"'''학도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n부리와 다리가 길..."
1,뒷부리도요,"'''뒷부리도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n부리가 길고 ..."
2,칼 스벤손,"'''칼 빌헬름 스벤손'''(, 1984년 3월 21일, 옌셰핑 주 옌셰핑 ~)은 ..."
3,노랑발도요,"'''노랑발도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n다리는 비교적..."
4,신창균,"'''신창균'''(申昌均, 1908년~2003년 3월 5일)은 대한민국의 독립운동이..."


In [6]:
# 데이터셋 라마인덱스 다큐먼트 오브젝트로 변환
docs = []

for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        extra_info={'title': row['title']}
    ))

In [7]:
docs

[Document(id_='79d63203-a445-43c8-850b-068302891618', embedding=None, metadata={'title': '학도요'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="'''학도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n부리와 다리가 길고 아랫부리의 절반이 붉은색이다. \n=== 여름깃 ===\n전체적으로 검은색이고 몸윗면에 흰 반점이 흩어져 있다. \n=== 겨울깃 ===\n전체적으로 회갈색이고 흰색 눈썹선이 명확하다. 몸아랫면은 흰색이며 가슴과 옆구리에 엷은 회갈색 기운이 있다.\n무논, 습지, 하구, 갯벌 등에서 무리지어 생활한다. 유라시아대륙 북부에서 번식하고 유럽 남부, 아프리카, 인도, 동남아시아에서 월동한다.\n분류:도요과\n분류:한국의 새\n분류:한국의 나그네새\n분류:유럽의 새\n분류:아프리카의 새\n분류:인도의 새\n분류:동남아시아의 새", path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='75d48578-8ff1-48fe-8231-6d6dcb2d13d9', embedding=None, metadata={'title': '뒷부리도요'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {

In [8]:
# 벡터스토어 인덱스 설정
vector_index = VectorStoreIndex.from_documents(
    docs,
    use_asnyc=True
    )

In [9]:
# 쿼리 엔진 설정
vector_query_engine = vector_index.as_query_engine(similarity_top_k=2)

- 구축된 Naive-RAG 파이프라인 이용한 기본적인 Q&A 테스팅

In [10]:
response = vector_query_engine.query('칼 스벤손이 누구야?')

In [11]:
# 검색에 대한 답변
response.response

'칼 스벤손은 스웨덴의 전직 프로 축구 선수로, 중앙 수비수로 활약한 선수이며, 스웨덴 국가대표팀에도 출전한 경력을 가지고 있습니다.'

In [12]:
# 소스노드 구조
response.source_nodes

[NodeWithScore(node=TextNode(id_='add3e3fb-f37f-46c0-b48c-a79df69dea02', embedding=None, metadata={'title': '칼 스벤손'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='68dd6a64-2783-4bf9-8b0a-c0e70f590a52', node_type='4', metadata={'title': '칼 스벤손'}, hash='2f3deb9488de6d2db9a41a7d91b49e9621685b43bfcd651af861efa7d9edad07'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='8690eda0-12be-4d5d-bd4d-0f1dc551025d', node_type='1', metadata={'title': '칼 스벤손'}, hash='d73b7e6831856b60cb82b3971af5b8af0883a64351fc4cbf194189cbf6f4cc81')}, metadata_template='{key}: {value}', metadata_separator='\n', text="스벤손은 2014년 수페레탄 시즌이 끝난 뒤 현역에서 은퇴했다.\nU-17, U-19, 그리고 U-21 연령대의 스웨덴 국가대표팀에서 활약한 스벤손은 2006년 1월에 사우디아라비아를 상대로 처음이자 마지막으로 스웨덴 국가대표팀 경기를 치렀다. 그는 2006년 월드컵에 스웨덴 선수단 일원으로 승선했지만, 한 경기도 출전하지 못했다.\n*  칼 스벤손 - 프랑스 프로축구연맹 \n*\n* #2 칼 스벤손 - ''ifkdb.com''\n분류:1984년 출생\n분류:살아있는 사람\n분류:스웨덴의 남자 축구 선수\n분류:남자 축구 센터백\n분류:스웨

In [13]:
print(response.source_nodes[0].score)
print(response.source_nodes[1].score)

0.48545534582170086
0.4756392517147872


In [14]:
# 참조 청크의 사이즈(글자수)
print(response.source_nodes[0].node.start_char_idx)
print(response.source_nodes[0].node.end_char_idx)
print(response.source_nodes[0].node.end_char_idx-response.source_nodes[0].node.start_char_idx)

815
1461
646


In [15]:
# 다른 질문
response = vector_query_engine.query('칼 스벤손이 언제 태어났고, 어떤 월드컵에 참가했지?')

In [16]:
response.response

'칼 스벤손은 1984년 3월 21일에 태어났고, 2006년 월드컵에 참가했습니다.'

In [17]:
# 소스노드 구조
response.source_nodes

[NodeWithScore(node=TextNode(id_='8690eda0-12be-4d5d-bd4d-0f1dc551025d', embedding=None, metadata={'title': '칼 스벤손'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='68dd6a64-2783-4bf9-8b0a-c0e70f590a52', node_type='4', metadata={'title': '칼 스벤손'}, hash='2f3deb9488de6d2db9a41a7d91b49e9621685b43bfcd651af861efa7d9edad07'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='add3e3fb-f37f-46c0-b48c-a79df69dea02', node_type='1', metadata={}, hash='86abd8eb7c0c274d8503f1dd1f0c68adb544b999039c534d13adb6d7da0d3eba')}, metadata_template='{key}: {value}', metadata_separator='\n', text="'''칼 빌헬름 스벤손'''(, 1984년 3월 21일, 옌셰핑 주 옌셰핑 ~)은 스웨덴의 전직 프로 축구 선수로, 현역 시절 옌셰핑스 셰드라, IFK, 레인저스, 그리고 캉에서 중앙 수비수로 활약했다. 그는 스웨덴 국가대표팀에 1번 출전했고, 2006년 월드컵에 자국을 대표로 출전했다.\n=== 초기 경력 ===\n칼 스벤손은 유년 시절에 속했던 옌셰핑스 셰드라 소속으로 프로 무대 신고식을 치럿고, 2003년 알스벤스칸 시즌을 앞두고 IFK로 이적했다. 그는 2004년 시즌에 IFK 일원으로 스벤스카 쿠펜 결승전에 올랐지만, 유르고르덴에 1-3으로 패했다. 스벤손은 이 결승

In [18]:
docs

[Document(id_='79d63203-a445-43c8-850b-068302891618', embedding=None, metadata={'title': '학도요'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="'''학도요'''(, )는 도요목 도요과에 속하는 나그네새이다.\n부리와 다리가 길고 아랫부리의 절반이 붉은색이다. \n=== 여름깃 ===\n전체적으로 검은색이고 몸윗면에 흰 반점이 흩어져 있다. \n=== 겨울깃 ===\n전체적으로 회갈색이고 흰색 눈썹선이 명확하다. 몸아랫면은 흰색이며 가슴과 옆구리에 엷은 회갈색 기운이 있다.\n무논, 습지, 하구, 갯벌 등에서 무리지어 생활한다. 유라시아대륙 북부에서 번식하고 유럽 남부, 아프리카, 인도, 동남아시아에서 월동한다.\n분류:도요과\n분류:한국의 새\n분류:한국의 나그네새\n분류:유럽의 새\n분류:아프리카의 새\n분류:인도의 새\n분류:동남아시아의 새", path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='75d48578-8ff1-48fe-8231-6d6dcb2d13d9', embedding=None, metadata={'title': '뒷부리도요'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {

In [19]:
# 다른 질문
response = vector_query_engine.query('신창균과 칼 스벤손에 대해서 알려줘')

In [20]:
response.response

'신창균과 칼 스벤손은 모두 축구 선수이다. 신창균에 대한 추가 정보는 제공되지 않았으나, 칼 스벤손은 스웨덴의 전직 프로 축구 선수로, 중앙 수비수로 활약했으며 스웨덴 국가대표팀에도 출전한 경력이 있다. 스벤손은 IFK, 레인저스, 캉 등에서 뛰었으며 프랑스와 스웨덴을 거쳐 현역에서 은퇴한 선수이다.'